# Training model

In [49]:
#Import Packages
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
import tqdm

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)


import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import seaborn as sns

df = pd.read_csv('Boliga - Final for training.csv')
df = df.drop(columns = ['Kommune'])
df_sample = df.sample(500)

In [50]:
df.shape

(61618, 36)

In [51]:
#Split into X and Y 
df_dum = pd.get_dummies(df_sample, drop_first = True, columns = ['isForeclosure', 'Type'])
X = df_dum.drop(columns = ['price'])
y = df_dum['price']

KeyError: "['Kommune'] not in index"

### Split data into Test, Development, Validation and Train

In [38]:
from sklearn.model_selection import train_test_split

X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size = 0.5, random_state = 1)

In [40]:
y_train

55519     2795000
25611     1250000
8355      2150000
44988     1000000
57635      520000
43478     3695000
52878     1945000
49577     2590000
23539     1945000
46006     1445000
36739     1895000
1001      2950000
29432     3450000
32129     1448000
5478      2995000
39312     2397000
37021     4860000
5669      3450000
14070     1875000
17137      595000
34699     1395000
43302     3295000
24310     1895000
27521     1195000
48611     3495000
5335       795000
59566     1450000
20062      149000
17310      745000
47563      895000
57659      645000
46861     1535000
1233      2195000
48169     1295000
2669      1670314
32679     1875000
25739     1595000
41042      750000
26116      695000
26711      589000
41768      845000
2687      1995000
10932     1995000
41719     1075000
948       2275000
59229     2395000
56221     1685000
48722     1650000
17049      525000
59901     1595000
57496      995000
54495      395000
31723     1095000
11503     2695000
38408     3185000
50577     

### Scale, fit and transform data

In [ ]:
from sklearn.preprocessing import StandardScaler

stdscale = StandardScaler(with_mean = 1, with_std = True)

#Scale explanatory variables
X_train_s = stdscale.fit_transform(X_train)
X_test_s = stdscale.transform(X_test)


#Scale target variable (because of regression model - no need when categorical)
y_train_s = stdscale.fit_transform(y_train)
y_test_s = stdscale.transform(y_test)


### Optimize hyperparameter with K-fold CV 

In [14]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None
df

price  basementSize  buildYear  Ejerudgift  isForeclosure  lotSize  \
0    18750000             0       1767       16872          False        0   
1    10500000            88       1880        5702          False        0   
2    15000000             0       1755        9486          False        0   
3    19995000             0       1880       17609          False        0   
4     8750000             0       1873        5590          False        0   
5    11500000             0       1873        5739          False        0   
6     4495000             0       1870        4461          False        0   
7     4494000             0       1872        9790          False        0   
8     9500000             0       1900        5606          False        0   
9     5995000             0       1796        2771          False        0   
10    7695000             0       1796        4723          False        0   
11    2795000             0       1797        2999          False        0   
12    2995000             0       1797        3248          False        0   
13    3500000             0       1797        3479          False        0   
14    3550000             0       1797        4509          False        0   
15    4195000             0       1799        4081          False        0   
16    4895000             0       1817        5087          False        0   
17    3395000             0       1690        2549          False        0   
18    5495000             0       1741        6834          False        0   
19    4995000             0       1848        5665          False        0   
20    4400000             0       1991        3401          False        0   
21   10995000             0       1810        7064          False        0   
22   23000000             0       1902       15908          False        0   
23    4595000             0       1902        2745          False        0   
24    4995000             0       1747        6041          False        0   
25    2995000             0       1734        1575          False        0   
26    3195000             0       1883        3703          False        0   
27    3550000             0       1848        3218          False        0   
28    3995000             0       1883        4572          False        0   
29    5295000             0       1877        5569          False        0   
30    5975000             0       1883        4811          False        0   
31    6495000             0       1700        2904          False        0   
32    7500000             0       1883        7966          False        0   
33    9295000             0       1802        5726          False        0   
34   11950000             0       1883        5273          False        0   
35    8995000             0       1730       11319          False        0   
36    8995000             0       1730       11319          False        0   
37    8995000             0       1900        5192          False        0   
38   11500000             0       1900        5362          False        0   
39    3495000             0       1735        2798          False        0   
40    6750000             0       1764        6043          False        0   
41    6195000             0       1847        3445          False        0   
42    4850000             0       1816        4814          False        0   
43    6500000             0       1930        2735          False        0   
44    7695000             0       1885        6289          False        0   
45    9800000             0       1930        4058          False        0   
46    5495000             0       1744        3661          False        0   
47    6345000             0       1744        3851          False        0   
48   11000000             0       1744       11869          False        0   
49   11500000             0       1744       11846          False        0   
50    3750000             0   

In [47]:
#Find optimized lambda
from sklearn.model_selection import KFold

lambdas =  np.logspace(-4, 4, 12) # define lambda - devide into 12 lambdas between 10**-4 and 10**4
kfolds = KFold(n_splits=5) # define nr. of folds

#Creates lists to append values of the loop:
mse_val_avr = []
mse_test_avr = []
lambda_list = []

#Outer-loop (Lambdas)
for lambda_ in tqdm.tqdm(lambdas):
#Inner-loop (Folds)
    mse_val_ = []
    mse_test_ = []
    #Split the dataset into 5 folds, and loop through each fold. 
    # In the iteration, the fold in question acts as the training-set. 
    for train_idx,val_idx in kfolds.split(X_dev, y_dev):
    # train model and compute MSE on test fold
        pipe_Lasso_CV = make_pipeline(PolynomialFeatures(include_bias=False, degree = 3), 
                        StandardScaler(with_mean = 1, with_std = True ),
                        Lasso(alpha = lambda_, random_state=1))
        # Assign X_train & y_train by extracting elements if the folds from X_dev and y_dev
        X_train, y_train = X_dev.iloc[train_idx], y_dev.iloc[train_idx]
        # Assign X_val & y_val by extracting elements of the folds from X_dev and y_dev
        X_val, y_val = X_dev.iloc[val_idx], y_dev.iloc[val_idx] 
        # Fit Lasso-model to training data
        pipe_Lasso_CV.fit(X_train, y_train)   
        #Calculate and append the mse into mse_val list:
        mse_val_.append(mse(pipe_Lasso_CV.predict(X_val), y_val))
        ##Calculate and append the mse into mse_test list:
        mse_test_.append(mse(pipe_Lasso_CV.predict(X_test), y_test))
        
    mse_val_avr.append(sum(mse_val_)/len(mse_val_))
    mse_test_avr.append(sum(mse_val_)/len(mse_val_))

df_val = pd.DataFrame(mse_val_avr, index = lambdas)
df_test = pd.DataFrame(mse_test_avr, index = lambdas)
df_join = pd.concat([df_test, df_val], axis=1)
df_with_index = df_join.reset_index()

#Convert to list
mses = df_with_index.values.tolist()



100%|██████████| 12/12 [03:06<00:00, 18.60s/it]

In [48]:
df_join

0             0
0.000100      2.542570e+13  2.542570e+13
0.000534      2.542397e+13  2.542397e+13
0.002848      2.541472e+13  2.541472e+13
0.015199      2.536542e+13  2.536542e+13
0.081113      2.510157e+13  2.510157e+13
0.432876      2.370881e+13  2.370881e+13
2.310130      1.748111e+13  1.748111e+13
12.328467     1.194811e+13  1.194811e+13
65.793322     8.290316e+12  8.290316e+12
351.119173    5.729008e+12  5.729008e+12
1873.817423   5.946784e+12  5.946784e+12
10000.000000  2.248274e+12  2.248274e+12